# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import nltk

from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 12345

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Лемматизация текста

In [5]:
corpus = data['text'].values.astype('U')

wnl = WordNetLemmatizer()

def clear_text(text):
    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = pattern.split()
    lemm = []
    for i in range(len(clear)):
        lemm.append(wnl.lemmatize(clear[i]))
    return " ".join(lemm)

for i in range(len(corpus)):
    corpus[i] = clear_text(corpus[i])
    
data_corpus = pd.DataFrame(corpus)
data['lemm_text'] = data_corpus[0]

In [6]:
data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [7]:
print(data['toxic'].value_counts())
balance = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
balance

0    143106
1     16186
Name: toxic, dtype: int64


8.841344371679229

Дисбаланс классов 1 к 9

## Обучение

Создание признаков

In [8]:
features = data['lemm_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE, stratify = target)

Определение стоп-слов и TFIDF-векторизация текста

In [9]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

### Логистическаая регрессия

In [11]:
lr_model = LogisticRegression()
lr_parameters = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10],
                'class_weight':['balanced'],
                'random_state':[RANDOM_STATE]}]
lr_grid = GridSearchCV(lr_model, lr_parameters, cv = 3, verbose=True,n_jobs=-1, 
                    scoring='f1')
lr_grid.fit(features_train, target_train)
print(lr_grid.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'C': 10, 'class_weight': 'balanced', 'random_state': 12345, 'solver': 'lbfgs'}


In [12]:
lr_model = LogisticRegression(fit_intercept=True, C = 10, class_weight = 'balanced', random_state = RANDOM_STATE, solver = 'newton-cg')
lr_model.fit(features_train, target_train)
lr_f1 = (cross_val_score(lr_model, features_train, target_train, cv = 3,
                           scoring='f1').mean())
print('LogisticRegression f1 =', lr_f1)

LogisticRegression f1 = 0.7601667703984704
CPU times: user 40.5 s, sys: 1min 10s, total: 1min 51s
Wall time: 1min 51s


### Дерево решений

In [13]:
dtr_model = DecisionTreeClassifier()
dtr_parameters = [{'max_depth':[5, 10, 15], 
                'min_samples_split':[10, 20, 30],
                'min_samples_leaf':[10, 30, 50],
                'class_weight':['balanced'],
                'random_state':[RANDOM_STATE]}]
dtr_grid = GridSearchCV(dtr_model, dtr_parameters, cv = 3, verbose=True,n_jobs=-1, 
                    scoring='f1')
dtr_grid.fit(features_train, target_train)
print(dtr_grid.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'class_weight': 'balanced', 'max_depth': 15, 'min_samples_leaf': 10, 'min_samples_split': 30, 'random_state': 12345}


In [14]:
dtr_model = DecisionTreeClassifier(max_depth = 15, min_samples_leaf = 10, min_samples_split = 30, 
                                   class_weight = 'balanced', random_state = RANDOM_STATE)
dtr_model.fit(features_train, target_train)
dtr_f1 = (cross_val_score(dtr_model, features_train, target_train, cv = 3,
                           scoring='f1').mean())
print('DecisionTreeRegressor f1 =', dtr_f1)

DecisionTreeRegressor f1 = 0.5755437313347758


### Случайный лес

In [15]:
rfr_model = RandomForestClassifier() 
rfr_parameters = [{'n_estimators':[100, 150, 200], 
                'max_depth':[5,10,15],
                'class_weight':['balanced'],
                'random_state':[RANDOM_STATE]}]
rfr_grid = GridSearchCV(rfr_model, rfr_parameters, cv = 3, verbose=True,n_jobs=-1, 
                    scoring='f1')
rfr_grid.fit(features_train, target_train)
print(rfr_grid.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'class_weight': 'balanced', 'max_depth': 15, 'n_estimators': 200, 'random_state': 12345}


In [16]:
rfr_model = RandomForestClassifier(n_estimators = 200, max_depth = 15, 
                                   class_weight = 'balanced', random_state = RANDOM_STATE)
rfr_model.fit(features_train, target_train)
rfr_f1 = (cross_val_score(rfr_model, features_train, target_train, cv = 3,
                           scoring='f1').mean())
print('RandomForestRegressor f1 =', rfr_f1)

RandomForestRegressor f1 = 0.3784147164097349


### Тестирование

In [17]:
lr_predictions_test = lr_model.predict(features_test)
print('f1:', f1_score(target_test, lr_predictions_test))

f1: 0.7601246105919003


In [18]:
dtr_predictions_test = dtr_model.predict(features_test)
print('f1:',f1_score(target_test, dtr_predictions_test))

f1: 0.5630630630630631


In [19]:
rfr_predictions_test = rfr_model.predict(features_test)
print('f1:',f1_score(target_test, rfr_predictions_test))

f1: 0.39369364260193956


## Выводы

Интернет-магазину «Викишоп» нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Для этого нужно обучить модель, которая будет классифицировать комментарии на позитивные и негативные. 

В распоряжение предоставлен набор данных с разметкой о токсичности комментариев.

Модель должна обладать значением метрики качества F1 не меньше 0.75.

В ходе работы было выполнено:
 - Загружены и изучены данные. Они содержат в себе комментарии пользователей к товарам, доступным для приобретения в интернет-магазине «Викишоп».
 - Над текстами комментариев была проведена лемматизация
 - Данные были разделены на обучающую и тестовые выборки
 - Обучено три модели: Логистическая регрессия, дерево решений и случайный лес
 - Модели протестированы на тестовой выборке

Для каждой модели проводился подбор гиперпараметров, с целью повышения метрики качества.

По результатам этого подбора, обучения и тестирования моделей, наилучший результат показала логистическаая регрессия. Так же это единственная модель, которая удовлетворяет условию о значении метрики качества f1. При этом данная модель работает гораздо быстрее, чем другие.

В связи с вышеизложенным, логистическая регрессия является лучшим решением для поставленной задачи.